## Hand base calibration
- the transformation of the allegro_base frame with respect to the iiwa_link_ee
- keep the iiwa on the Position model and read the ee pose
- attach a marker frame on the thumb tip

In [1]:
import time

import numpy as np
from controller_utils import Robot
import rospy
import tools.rotations as rot
%load_ext autoreload
%autoreload 2


In [2]:
optitrack_frame_names=['iiwa_base7', 'thumb_tip_m']
r = Robot(optitrack_frame_names=optitrack_frame_names, calibration=True)

Kinematic model has been loaded from kinematics/q2pose_left.txt


Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_0']/collision[1]
Unknown tag "self_collision_checking" in /robot[@name='iiwa7']/link[@name='iiwa_link_0']
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_1']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_2']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_3']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_4']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_5']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_6']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_7']/collision[1]
Unknown tag "hardwareInterface" in /robot[@name='iiwa7']/transmission[@name='iiwa_tran_1']/actuator[@name='iiwa_motor_1']
Unknown tag "robotNamespace" in /robot[@name='iiwa7']/transmission[@name='iiwa_tran_1']
Unkno

In [5]:
print(r.x)
print(r.x_obj)
print(r.xh)
# print(r.q) # [ 0.31333505  0.91675294  0.38246485 -1.1181452   1.05204611 -0.18829075  -2.78992007] for calibration of the hand

[ 0.69128507  0.37766046  0.33675591 -0.4752421   0.70036868 -0.46459103
  0.2603533 ]
{'thumb_tip_m': array([0.83429761, 0.40487476, 0.37016009, 0.57427467, 0.32517999,
       0.1313109 , 0.73974591]), 'iiwa_base7': array([ 3.10812672e-02,  6.80085266e-04,  7.65026371e-02,  9.98941116e-01,
        4.30501745e-03, -8.87875679e-03, -4.49364128e-02])}
[array([-0.01729725, -0.05141172,  0.23178008,  0.99319922,  0.04789255,
        0.09693565, -0.04318681]), array([-4.25452266e-03,  9.43101509e-04,  2.36642788e-01,  9.80293493e-01,
        4.79786290e-03,  1.96025420e-01, -2.39933871e-02]), array([ 0.02733789,  0.05361915,  0.22799123,  0.89370769, -0.02895118,
        0.44595992, -0.03959982]), array([ 0.04052182,  0.05830514,  0.09662654,  0.04850917,  0.0933361 ,
       -0.40760921, -0.9070777 ])]


In [4]:
q1 = np.array([ 0.31333505 , 0.91675294 , 0.38246485, -1.1181452  , 1.05204611, -0.18829075 , -2.78992007])
r.move_to_joints(q1)

Linear interpolation by 1446 joints
Trajectory has been executed.


In [6]:
nums = 40
import time

q_rand = np.random.uniform(r.hand.bounds[0, 12:], r.hand.bounds[1, 12:], size=(nums, 4))
q0 = np.copy(r.qh)
pose_list = []
x_identity = np.array([0,0,0,1,0,0,0])
# x0 = np.copy(r.x)
for i in range(nums):
    q_cmd = np.concatenate([q0[:12], q_rand[i,:]])  # only control the thumb
    assert q_cmd.shape == (16,)
    r.move_to_joints(q_cmd)
    print(i)
    time.sleep(2)
    pose_list.append([[r.x, r.x_obj['thumb_tip_m']], r.xh[-1]]) # todo, check if the pose is valid or has been updated, otherwise might some markers are not seen.

Linear interpolation by 345 joints
Trajectory has been executed.
0
Linear interpolation by 174 joints
Trajectory has been executed.
1
Linear interpolation by 51 joints
Trajectory has been executed.
2
Linear interpolation by 53 joints
Trajectory has been executed.
3
Linear interpolation by 137 joints
Trajectory has been executed.
4
Linear interpolation by 318 joints
Trajectory has been executed.
5
Linear interpolation by 234 joints
Trajectory has been executed.
6
Linear interpolation by 212 joints
Trajectory has been executed.
7
Linear interpolation by 175 joints
Trajectory has been executed.
8
Linear interpolation by 121 joints
Trajectory has been executed.
9
Linear interpolation by 108 joints
Trajectory has been executed.
10
Linear interpolation by 288 joints
Trajectory has been executed.
11
Linear interpolation by 202 joints
Trajectory has been executed.
12
Linear interpolation by 279 joints
Trajectory has been executed.
13
Linear interpolation by 164 joints
Trajectory has been execu

In [7]:
#Solve the AX=XB problem.
from Pose_Estimation_Class import EKF, Batch_Processing, IEKF, UKF

num=20
A = np.zeros([4,4,int(nums/2)])
B = np.zeros([4,4,int(nums/2)])
for i in range(0, nums, 2):
    T_m12w_1 = rot.pose2T(pose_list[i][0][0] )
    T_m22w_1 = rot.pose2T(pose_list[i][0][1] )
    T_base2fk_1 = np.linalg.inv(rot.pose2T(pose_list[i][1] ))

    T_m12w_2 = rot.pose2T(pose_list[i+1][0][0] )
    T_m22w_2 = rot.pose2T(pose_list[i+1][0][1] )
    T_base2fk_2 = np.linalg.inv(rot.pose2T(pose_list[i+1][1] ))


    A[:,:,int(i/2)] = np.linalg.inv(T_m12w_2) @ T_m22w_2 @ np.linalg.inv(T_m22w_1) @ T_m12w_1
    B[:,:,int(i/2)] = np.linalg.inv(T_base2fk_2) @ T_base2fk_1

#Solve the AX=XB problem.
ekf=EKF()
for i in range(len(A[1,1,:])):
    AA=A[:,:,i]
    BB= A[:,:,i]
    ekf.Update(AA,BB)


iekf=IEKF()
for i in range(len(A[1,1,:])):
    AA=A[:,:,i]
    BB=B[:,:,i]
    iekf.Update(AA,BB)


ukf=UKF()
for i in range(len(A[1,1,:])):
    AA=A[:,:,i]
    BB=B[:,:,i]
    ukf.Update(AA,BB)

#Solve the AX=YB problem.
A = np.zeros([4,4,nums])
B = np.zeros([4,4,nums])
# [r.x, r.x_obj['thumb_tip_m']], r.xh[-1]
for i in range(0, nums):
    T_m12w_1 = rot.pose2T(pose_list[i][0][0] )
    T_m22w_1 = rot.pose2T(pose_list[i][0][1] )
    T_base2fk_1 = np.linalg.inv(rot.pose2T(pose_list[i][1] ))
    A[:,:,i] = np.linalg.inv(T_m22w_1) @ T_m12w_1  # T_ee2marker_m
    B[:,:,i] = T_base2fk_1
X_est,Y_est,Y_est_check,ErrorStats=Batch_Processing.pose_estimation(A, B)
x_est = rot.T2pose(X_est) # [x,y,z, qw, qx, qy, qz]

In [8]:
print('position:\n', x_est[:3], '\n', ekf.x[3:], '\n', iekf.x[3:], '\n', ukf.x[3:], '\n' )
print('quaternion:\n', x_est[3:], '\n', rot.axisangle2quat(ekf.x[:3]),'\n',  rot.axisangle2quat(iekf.x[:3]), '\n', rot.axisangle2quat(ukf.x[:3]), '\n')
print('quat error in degree:\n',
      rot.ori_dis(x_est[3:], rot.axisangle2quat(ekf.x[:3]))*180/np.pi,
        rot.ori_dis(x_est[3:], rot.axisangle2quat(iekf.x[:3]))*180/np.pi,
        rot.ori_dis(x_est[3:], rot.axisangle2quat(ukf.x[:3]))*180/np.pi)

position:
 [ 0.02261976 -0.02329719  0.02584867] 
 [-0.00340164 -0.00585751 -0.01103241] 
 [-0.00747859 -0.05007174  0.02277088] 
 [-0.02036417 -0.0435866   0.04955417] 

quaternion:
 [ 0.7120723   0.00295735 -0.02026721 -0.70180734] 
 [ 0.99846416 -0.02298539  0.05033421  0.00273081] 
 [ 0.73774093 -0.07730777  0.48783535 -0.46019399] 
 [ 0.84323123  0.05128614  0.31699969 -0.43109399] 

quat error in degree:
 89.85936598168682 66.10383615750698 52.5412454599508


In [11]:
np.savetxt('description/config/allegro_left_base_2_iiwa_link_ee.txt', x_est)

In [12]:
rot.quat2euler(x_est[3:])

array([-0.02425126, -0.03302043, -1.55667676])

In [21]:
rot.quat2euler(x_est[3:]) * 180/np.pi

array([ -1.38949481,  -1.89193104, -89.19100855])

In [15]:
T_palm2base = np.array([0.015,0,0.1075, 1,0,0,0])
T_root2palm = np.array([0,0,-0.095,1,0,0,0])

T_root2base = rot.pose_mul(T_palm2base, T_root2palm)

T_root_2ee = rot.pose_mul(x_est, T_root2base)

In [19]:
print(T_root2base)

[0.015  0.     0.0125 1.     0.     0.     0.    ]


In [17]:
print(T_root_2ee)

[ 0.02241875 -0.03798817  0.03870887  0.7120723   0.00295735 -0.02026721
 -0.70180734]
